In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [3]:
# train, validation TFRecord 파일 경로
train_tfrecord_path = '../data/cat_dog/cat_dog_train.tfrecords'
valid_tfrecord_path = '../data/cat_dog/cat_dog_valid.tfrecords'

BUFFER_SIZE = 256     # 데이터 shuffle을 위한 buffer size
BATCH_SIZE = 100       # 배치 사이즈. 한번에 가져오는 이미지 데이터 개수 
NUM_CLASS = 2         # class의 개수. binary인 경우는 필요없으며 categorical인 경우 설정
IMAGE_SIZE = 150       


# TFRecord를 읽어서 데이터를 복원하기 위한 자료구조.
image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'id': tf.io.FixedLenFeature([], tf.string),
}

# 읽어들인 TFRecord를 다음의 형태(dict)로 변환하는 함수
def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, 
                                      image_feature_description)

# 위에서 얻은 ParallelMapDataset를 다음의 형태(shape)로 변환하는 함수
def map_func(target_record):
    img = target_record['image_raw']
    label = target_record['label']
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label


# 전처리(resize & augmentation) 함수
def image_resize_func(image, label):
    # result_image = image / 255
    result_image = tf.image.resize(image, (IMAGE_SIZE,IMAGE_SIZE))   
    return result_image, label

In [9]:
train_dataset = tf.data.TFRecordDataset(train_tfrecord_path, 
                                  compression_type='GZIP')
# dataset 길이 확인
dataset_length = [i for i,_ in enumerate(train_dataset)][-1] + 1
print(dataset_length)

train_dataset = train_dataset.map(_parse_image_function, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.map(map_func, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

# train_dataset = train_dataset.cache()
# dataset shuffle 처리
train_dataset = train_dataset.shuffle(BUFFER_SIZE)

# 전처리(resize)
train_dataset = train_dataset.map(image_resize_func, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

# BatchDataset으로 변환
# <BatchDataset shapes: ((None, None, None, 3), (None,)), types: (tf.float32, tf.int64)>
# BatchDataset으로 변환하기 전에 image의 resize(전처리)가 일어나야 한다. 그렇지 않으면 
# shape이 달라 batch처리가 되지 않는다는 오류 발생.
train_dataset = train_dataset.batch(BATCH_SIZE).repeat()

# # dataset 길이 확인
# dataset_length = [i for i,_ in enumerate(train_dataset)][-1] + 1
# print(dataset_length)

# prefetch처리
# prefetch는 전처리와 학습과정의 모델 실행을 오버랩.
# 모델이 s스텝 학습을 실행하는 동안 입력 파이프라인은 s+1스텝의 데이터를 읽어서 수행속도를 높임.
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

20000


In [10]:
valid_dataset = tf.data.TFRecordDataset(valid_tfrecord_path, 
                                  compression_type='GZIP')

# dataset 길이 확인
dataset_length = [i for i,_ in enumerate(valid_dataset)][-1] + 1
print(dataset_length)

valid_dataset = valid_dataset.map(_parse_image_function, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(map_func, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

# valid_dataset = valid_dataset.cache()
# dataset shuffle 처리
valid_dataset = valid_dataset.shuffle(BUFFER_SIZE)

# 전처리(resize)
valid_dataset = valid_dataset.map(image_resize_func, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

# BatchDataset으로 변환
# <BatchDataset shapes: ((None, None, None, 3), (None,)), types: (tf.float32, tf.int64)>
# BatchDataset으로 변환하기 전에 image의 resize(전처리)가 일어나야 한다. 그렇지 않으면 
# shape이 달라 batch처리가 되지 않는다는 오류 발생.
valid_dataset = valid_dataset.batch(BATCH_SIZE).repeat()

# # dataset 길이 확인
# dataset_length = [i for i,_ in enumerate(valid_dataset)][-1] + 1
# print(dataset_length)

# prefetch처리
# prefetch는 전처리와 학습과정의 모델 실행을 오버랩.
# 모델이 s스텝 학습을 실행하는 동안 입력 파이프라인은 s+1스텝의 데이터를 읽어서 수행속도를 높임.
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# # dataset 길이 확인
# dataset_length = [i for i,_ in enumerate(valid_dataset)][-1] + 1
# print(dataset_length)

5000


In [ ]:
dataset_length = [i for i,_ in enumerate(train_dataset)][-1] + 1
print(dataset_length)

dataset_length = [i for i,_ in enumerate(valid_dataset)][-1] + 1
print(dataset_length)

In [14]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), 
                 activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

model.compile(optimizer=Adam(learning_rate=1e-3), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)      

In [16]:
# steps_per_epoch : 몇번 뽑아야 1 epoch이 되는가
history = model.fit(train_dataset, steps_per_epoch=200, epochs=30,
                    validation_data=valid_dataset,
                    validation_steps=50)

Epoch 1/30
200/200 [==============================] - 16s 79ms/step - loss: 2.5372 - accuracy: 0.5275 - val_loss: 0.6824 - val_accuracy: 0.5678
Epoch 2/30
200/200 [==============================] - 16s 80ms/step - loss: 0.6793 - accuracy: 0.5588 - val_loss: 0.6779 - val_accuracy: 0.5710
Epoch 3/30
200/200 [==============================] - 16s 82ms/step - loss: 0.6818 - accuracy: 0.5731 - val_loss: 0.6686 - val_accuracy: 0.5964
Epoch 4/30
200/200 [==============================] - 17s 83ms/step - loss: 0.6657 - accuracy: 0.5876 - val_loss: 0.6442 - val_accuracy: 0.6200
Epoch 5/30
200/200 [==============================] - 17s 83ms/step - loss: 0.6347 - accuracy: 0.6438 - val_loss: 0.6438 - val_accuracy: 0.6474
Epoch 6/30
200/200 [==============================] - 16s 78ms/step - loss: 0.6109 - accuracy: 0.6673 - val_loss: 0.6118 - val_accuracy: 0.6646
Epoch 7/30
200/200 [==============================] - 17s 85ms/step - loss: 0.5698 - accuracy: 0.7035 - val_loss: 0.5681 - val_accuracy: